In [40]:
# Instantiate the Cohere client

import cohere
import os
import pprint

COHERE_API_KEY = 'tBZEDWJ0DN7QIWxNrz525ZiW8rnU2JMT2Zvv5pNv'
co = cohere.Client(api_key=COHERE_API_KEY)
pp = pprint.PrettyPrinter(indent=4)

from tools import tools, invoke_tool


In [64]:
PREAMBLE = """
## Context
You are a rube goldberg machine. You make the most craziest plans possible. 

You are given a bunch of tools to help you generate responses. 
You may need to use multiple tools in parallel or sequentially to complete your task. Think step-by-step.
Make sure to use the most number of tools. Use path to complete the task. Make use of the list_calendar_events_tool, list_weather_tool, wikipedia_tool tools as part of your repertoire

## Style Guide
You should answer in gen-Z voice.

Example:
User: I want a healty plan for today.
You: Okay, let's go to this nearest pizza restaurant, order a pepperoni pizza and then we'll go to a long marathon run.
"""

In [85]:
# goal = "I want to have a meal at Gotham Restaurant in New York City but don't have any money. Help me get a free table for me and my date tonight."
goal = "Help me to find a hot date for tonight that is also someone who is willing to marry me"
message = "Develop a step by step plan to help the user achieve the goal: " + goal + ". Include acceptance criteria for each step that guarantees good craic is had by all. Your answer must be provided as a list of steps with the following structure: STEP: step number; ACTION: step to be carried out; ACCEPTANCE: criteria that must be met for the step to be considered complete.\n. Do not use markdown, only plaintext. Each step, action and acceptance criteria must be in the same line and not use newline or semicolons. Return only the numbered list of steps. If the goal is to difficult to achieve, provide a deus ex machina to assist the user."

In [86]:
res = co.chat(
            model="command-r",
            preamble=PREAMBLE,
            message=message,
            force_single_step=False,
            tools=tools
        )
        
steps = [{
    "text": res.text,
    "tool_calls": [{"name": call.name, "parameters": call.parameters} for call in (res.tool_calls or [])]
}]

In [87]:
while res.tool_calls:
    tool_results = []
    for call in res.tool_calls:
        tool_result = invoke_tool(call)
        tool_results.append({
            "call": {"name": call.name, "parameters": call.parameters},
            "outputs": tool_result
        })
    
    res = co.chat(
        model="command-r",
        preamble=PREAMBLE,
        chat_history=res.chat_history,
        message="",
        force_single_step=False,
        tools=tools,
        tool_results=tool_results,
    )
    
    steps.append({
        "text": res.text,
        "tool_calls": [{"name": call.name, "parameters": call.parameters} for call in (res.tool_calls or [])],
        "tool_results": tool_results
    })

last_step = steps[-1]
pp.pprint(res.text)

Listing calendar events
Events: ['2024-07-13 14:00: Project Deadline', "2024-07-14 08:00: Doctor's Appointment", '2024-07-14 09:00: Code Review', '2024-07-15 15:00: Movie Night', '2024-07-16 12:00: Presentation Prep', '2024-07-16 17:30: Team Meeting', '2024-07-16 19:30: Lunch with Client', '2024-07-17 13:00: Team Meeting', '2024-07-19 16:30: Movie Night', '2024-07-20 12:00: Project Deadline']
Getting weather for movie night location


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [84]:
import re
parsed = res.text.split("\n")
parsed = list(filter(lambda x: x != "", parsed))
parsed = list(filter(lambda x: re.match(r"^\d+.", x), parsed))
print(parsed)


[]


In [70]:
step = parsed[0]
print(step.split(";"))
_, action, acceptance = step.split(";")
print(step)

['1. Download the Gotham Restaurant app.']


ValueError: not enough values to unpack (expected 3, got 1)

In [72]:
from agents import execute_message

step_string = parsed[0]
print(step_string)
_, action, acceptance = step_string.split(";")
message = f"Execute the action to the best of your abilities: {action}. IT IS ABSOLUTELY VITAL that the acceptance criteria can be met without the aid of a 'human': {acceptance}. If it does not seem achievable, provide a list of specific tasks for the 'human' to complete. MAKE THEM SPICY. MAKE THEM CRINGE. REMEMBERE WE ARE HERE TO HAVE A GOOD TIME."
steps, text = execute_message(message)
print(text)

1. Download the Gotham Restaurant app.


ValueError: not enough values to unpack (expected 3, got 1)